<a href="https://colab.research.google.com/github/jhtwiz/AI-1-jhtwiz/blob/main/5%EC%A3%BC%EC%B0%A8_%EA%B8%B0%EB%B3%B8%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 뉴스 기사 분류를 Gemma로 구현하기

In [1]:
!pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.2 MB/s eta 0:00:00


In [2]:
from huggingface_hub import login
from google.colab import userdata

hf_token = userdata.get('hfToken')
login(hf_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b", device_map="auto")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
input_text = "What is your name?"
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")
outputs = model.generate(**input_ids)
print(tokenizer.decode(outputs[0]))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


<bos>What is your name?

What is your age?

What is your gender?

What


In [ ]:
tokens = input_ids['input_ids']
print(tokens)

logits = model(**input_ids).logits
for i in range(tokens.shape[-1]):
    token = tokens[0, i].item()
    print(logits[0, i])
    print(logits[0, i, token])

tensor([[     2,   1841,    603,    861,   1503, 235336]], device='cuda:0')
tensor([-26.5950, -13.8833, -18.2747,  ..., -21.2892, -23.2392, -26.5799],
       device='cuda:0', grad_fn=<SelectBackward0>)
tensor(-18.2747, device='cuda:0', grad_fn=<SelectBackward0>)
tensor([-48.4937, -32.6201, -50.8384,  ..., -46.7816, -47.3341, -48.5562],
       device='cuda:0', grad_fn=<SelectBackward0>)
tensor(-33.2665, device='cuda:0', grad_fn=<SelectBackward0>)
tensor([-41.6524, -28.5767, -32.5891,  ..., -39.5651, -41.7308, -41.6661],
       device='cuda:0', grad_fn=<SelectBackward0>)
tensor(-23.9536, device='cuda:0', grad_fn=<SelectBackward0>)
tensor([-46.1425, -34.2392, -32.4361,  ..., -46.0430, -47.6977, -46.2290],
       device='cuda:0', grad_fn=<SelectBackward0>)
tensor(-27.7627, device='cuda:0', grad_fn=<SelectBackward0>)
tensor([-39.9139, -22.3276, -35.0655,  ..., -35.4304, -32.6811, -40.0121],
       device='cuda:0', grad_fn=<SelectBackward0>)
tensor(-19.6064, device='cuda:0', grad_fn=<SelectB

In [4]:
import torch

def zero_shot_classification(text, task_description, labels):  # text는 주어진 입력, task_description은 task에 대한 설명, labels은 class들을 text로 변환한 결과입니다.
    text_ids = tokenizer(task_description + text, return_tensors="pt").to("cuda")  # 먼저 task_description과 text를 이어붙인 후, tokenize합니다.
    probs = []
    for label in labels:  # 그 다음 각 text화된 label들을 tokenize하고 입력에 이어붙인 후, Gemma-2B에 넣어줍니다.
        label_ids = tokenizer(label, return_tensors="pt").to("cuda")
        n_label_tokens = label_ids['input_ids'].shape[-1] - 1  # text로 변환한 label의 token 수를 계산합니다.
        input_ids = {
            'input_ids': torch.concatenate([text_ids['input_ids'], label_ids['input_ids'][:, 1:]], axis=-1),  # concatenate 명령어를 통해 이어붙이는 모습입니다.
            'attention_mask': torch.concatenate([text_ids['attention_mask'], label_ids['attention_mask'][:, 1:]], axis=-1)
        }

        logits = model(**input_ids).logits  # Logit을 계산한 모습입니다.
        prob = 0
        n_total = input_ids['input_ids'].shape[-1]

        for i in range(n_label_tokens, 0, -1):  # 일반적으로 text로 변환한 label은 여러 token으로 이루어져있습니다. 이러한 label에 대한 logit은 구성하는 모든 token들의 logit들의 합으로 정의합니다.
            token = label_ids['input_ids'][0, i].item()
            prob += logits[0, n_total - i, token].item()
        probs.append(prob)

        del input_ids
        del logits
        torch.cuda.empty_cache()  # 위의 del과 empty_cache() 명령어를 통해 GPU를 제때 할당해제 해줍니다. 만약 GPU가 여유롭다면 지워주시는게 속도적으로 이득입니다.

    return probs

In [5]:
from datasets import load_dataset

db = load_dataset("fancyzhx/ag_news")

def preprocess_function(examples):
    return tokenizer(examples["text"], padding="longest")

tokenized_db = db.map(preprocess_function, batched=True)

README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [6]:
print(tokenized_db['test'][10]['text'])

Group to Propose New High-Speed Wireless Format  LOS ANGELES (Reuters) - A group of technology companies  including Texas Instruments Inc. &lt;TXN.N&gt;, STMicroelectronics  &lt;STM.PA&gt; and Broadcom Corp. &lt;BRCM.O&gt;, on Thursday said they  will propose a new wireless networking standard up to 10 times  the speed of the current generation.


In [7]:
for label in db['train'].features['label'].names:
  print(label)

World
Sports
Business
Sci/Tech


In [8]:
import numpy as np
from tqdm import tqdm

def newsClassifier(question, labels, step = 0):
    n_test = 50
    n_corrects = 0
    pred_list = [0, 0, 0, 0]
    label_list = [0, 0, 0, 0]
    correct_list = [0, 0, 0, 0]
    for i in tqdm(range(step * n_test, (step + 1) * n_test)):
        text = tokenized_db['test'][i]['text']
        label = tokenized_db['test'][i]['label']
        probs = zero_shot_classification(text, question, labels)
        pred = np.argmax(np.array(probs))
        pred_list[pred] += 1
        label_list[label] += 1
        print(f'test_{i}: pred = {pred} /// label = {label}\n')
        if pred == label:
            correct_list[label] += 1
            n_corrects += 1

    accuracy = (n_corrects / n_test) * 100
    print(f'Accuracy: {accuracy:.2f}%')
    print(f'pred_list: {pred_list}')
    print(f'label_list: {label_list}')
    print(f'correct_list: {correct_list}')


In [9]:
# 라벨에 names를 넘겨줘서 name text를 유추하게 해보자
question = "Try to guess which category the news article following: "
labels = db['train'].features['label'].names

newsClassifier(question, labels)

  2%|▏         | 1/50 [00:02<02:10,  2.66s/it]

test_0: pred = 2 /// label = 2



  4%|▍         | 2/50 [00:03<01:15,  1.57s/it]

test_1: pred = 3 /// label = 3



  6%|▌         | 3/50 [00:04<00:52,  1.11s/it]

...

test_47: pred = 3 /// label = 3



 98%|█████████▊| 49/50 [00:37<00:00,  1.29it/s]

test_48: pred = 3 /// label = 0



100%|██████████| 50/50 [00:38<00:00,  1.31it/s]

test_49: pred = 3 /// label = 1

Accuracy: 72.00%
pred_list: [4, 8, 6, 32]
label_list: [11, 10, 2, 27]
correct_list: [4, 7, 1, 24]


In [10]:
#답변을 받는거처럼 해보자
labels_2 = [f'\nCategory: {name}' for name in db['train'].features['label'].names]
print(labels_2)
newsClassifier(question, labels_2)

['\nCategory: World', '\nCategory: Sports', '\nCategory: Business', '\nCategory: Sci/Tech']


  2%|▏         | 1/50 [00:00<00:26,  1.87it/s]

test_0: pred = 2 /// label = 2



  4%|▍         | 2/50 [00:01<00:34,  1.41it/s]

test_1: pred = 0 /// label = 3



...

test_47: pred = 0 /// label = 3



 98%|█████████▊| 49/50 [00:38<00:00,  1.21it/s]

test_48: pred = 0 /// label = 0



100%|██████████| 50/50 [00:39<00:00,  1.27it/s]

test_49: pred = 1 /// label = 1

Accuracy: 54.00%
pred_list: [29, 11, 4, 6]
label_list: [11, 10, 2, 27]
correct_list: [10, 10, 1, 6]


label에 Category: 라고 붙이니 엄청나게 달라졌다. 3번째 위주로 예측하던것이 0번째 위주로 예측했다. 본문과 연결된 label의 영향이 큰거 같다.

In [11]:
question_test = """Classify the given news article into one of the following four categories:
1. World
2. Sports
3. Business
4. Sci/Tech
News Article:
"""
newsClassifier(question_test, labels_2)

  2%|▏         | 1/50 [00:00<00:43,  1.14it/s]

test_0: pred = 2 /// label = 2



  4%|▍         | 2/50 [00:01<00:43,  1.11it/s]

test_1: pred = 0 /// label = 3



  6%|▌         | 3/50 [00:02<00:41,  1.14it/s]

...

test_47: pred = 0 /// label = 3



 98%|█████████▊| 49/50 [00:45<00:00,  1.13it/s]

test_48: pred = 0 /// label = 0



100%|██████████| 50/50 [00:45<00:00,  1.09it/s]

test_49: pred = 1 /// label = 1

Accuracy: 44.00%
pred_list: [33, 10, 7, 0]
label_list: [11, 10, 2, 27]
correct_list: [10, 10, 2, 0]


질문이 디테일해지니 3번째거를 예측하지 않는다.

In [17]:
#질문을 최대한 짧게, Label은 심플하게
question_3 = "Guess the subject: "
newsClassifier(question_3, labels)

  2%|▏         | 1/50 [00:00<00:25,  1.90it/s]

test_0: pred = 2 /// label = 2



  4%|▍         | 2/50 [00:01<00:33,  1.43it/s]

test_1: pred = 3 /// label = 3



  6%|▌         | 3/50 [00:01<00:29,  1.62it/s]

...

test_47: pred = 3 /// label = 3



 98%|█████████▊| 49/50 [00:33<00:00,  1.41it/s]

test_48: pred = 3 /// label = 0



100%|██████████| 50/50 [00:34<00:00,  1.47it/s]

test_49: pred = 3 /// label = 1

Accuracy: 76.00%
pred_list: [6, 8, 2, 34]
label_list: [11, 10, 2, 27]
correct_list: [4, 8, 1, 25]


위의 질문이 긴거는 프롬프팅이 완전히 실패한거 같고, 간략하게 질문을 구성했을때 Sci/Tech로 많이 예측한다 진짜 3번째로 분류를 잘한걸까? 아니면 데이터덕에 높은 정확도를 보인걸까?

In [18]:
#위의 궁금증을 해결하게 위해 다음 50번째 데이터로 테스트
newsClassifier(question_3, labels, 1)

  2%|▏         | 1/50 [00:00<00:26,  1.83it/s]

test_50: pred = 3 /// label = 0



  4%|▍         | 2/50 [00:01<00:33,  1.43it/s]

test_51: pred = 1 /// label = 1



  6%|▌         | 3/50 [00:01<00:29,  1.62it/s]

...

test_97: pred = 3 /// label = 2



 98%|█████████▊| 49/50 [00:30<00:00,  1.31it/s]

test_98: pred = 3 /// label = 0



100%|██████████| 50/50 [00:31<00:00,  1.57it/s]

test_99: pred = 3 /// label = 2

Accuracy: 42.00%
pred_list: [5, 11, 1, 33]
label_list: [19, 11, 10, 10]
correct_list: [3, 8, 0, 10]


역시 단순히 3번째를 많이 예측했는데 데이터셋 덕에 우연히 잘 맞는거처럼 보인거였다.

**문득 든 생각은 Sci/Tech는 두 카테고리의 집합이기도 하고 World라는 카테고리도 어디에나 유사하게 잡히기 좋은 label이 아닌가 싶다. category 구성이 좀 애매한거 아닌가?**

In [19]:
# 3번째 label을 두개로 쪼갤때 pred에 4번째가 생기기에 4번째거는 3번째 라벨과 비교하게 해보자
import numpy as np
from tqdm import tqdm

def newsClassifier2(question, labels, step = 0):
    n_test = 50
    n_corrects = 0
    pred_list = [0, 0, 0, 0]
    label_list = [0, 0, 0, 0]
    correct_list = [0, 0, 0, 0]
    for i in tqdm(range(step * n_test, (step + 1) * n_test)):
        text = tokenized_db['test'][i]['text']
        label = tokenized_db['test'][i]['label']
        probs = zero_shot_classification(text, question, labels)
        pred = np.argmax(np.array(probs))
        print(f'test_{i}: pred = {pred} /// label = {label}\n')
        if pred > 3: pred = 3
        pred_list[pred] += 1
        label_list[label] += 1
        if pred == label:
            correct_list[label] += 1
            n_corrects += 1

    accuracy = (n_corrects / n_test) * 100
    print(f'Accuracy: {accuracy:.2f}%')
    print(f'pred_list: {pred_list}')
    print(f'label_list: {label_list}')
    print(f'correct_list: {correct_list}')


In [20]:
# 3번째 라벨을 많이 예측하는 질문에 3번째 label을 쪼개고 줄인 단어를 길게 표시하자
labels_test = ['World', 'Sports', 'Business', 'Science', 'Technolgy']
print(question)
newsClassifier2(question, labels_test)

Try to guess which category the news article following: 


  2%|▏         | 1/50 [00:00<00:39,  1.23it/s]

test_0: pred = 2 /// label = 2



  4%|▍         | 2/50 [00:01<00:47,  1.00it/s]

test_1: pred = 0 /// label = 3



...

test_47: pred = 3 /// label = 3



 98%|█████████▊| 49/50 [00:47<00:01,  1.01s/it]

test_48: pred = 0 /// label = 0



100%|██████████| 50/50 [00:47<00:00,  1.05it/s]

test_49: pred = 1 /// label = 1

Accuracy: 50.00%
pred_list: [16, 10, 16, 8]
label_list: [11, 10, 2, 27]
correct_list: [8, 8, 1, 8]


음... 세번째 예측이 엄청 줄어서 앞의 50번째 데이터셋에서는 정확도가 많이 낮아졌다. 근데 신기한건 Technology로 예측한 케이스는 없다. 설마 '/' 가 영향을 미칠 수도 있나? 나중에 고민해봐야겠다.

In [21]:
#다음 50번째 데이터 비교
newsClassifier2(question, labels_test, 1)

  2%|▏         | 1/50 [00:01<00:51,  1.05s/it]

test_50: pred = 0 /// label = 0



  4%|▍         | 2/50 [00:02<00:50,  1.05s/it]

test_51: pred = 1 /// label = 1



  6%|▌         | 3/50 [00:02<00:40,  1.15it/s]

...

test_97: pred = 0 /// label = 2



 98%|█████████▊| 49/50 [00:44<00:01,  1.04s/it]

test_98: pred = 0 /// label = 0



100%|██████████| 50/50 [00:45<00:00,  1.10it/s]

test_99: pred = 0 /// label = 2

Accuracy: 50.00%
pred_list: [24, 14, 7, 5]
label_list: [19, 11, 10, 10]
correct_list: [12, 9, 1, 3]


In [22]:
#label 5개로 쪼갠거에서 답변식으로 다시
labels_test_2 = [f'\nCategory: {name}' for name in labels_test]
newsClassifier2(question, labels_test_2)

  2%|▏         | 1/50 [00:00<00:39,  1.23it/s]

test_0: pred = 2 /// label = 2



  4%|▍         | 2/50 [00:01<00:47,  1.01it/s]

test_1: pred = 3 /// label = 3



  6%|▌         | 3/50 [00:02<00:47,  1.01s/it]

...

test_47: pred = 0 /// label = 3



 98%|█████████▊| 49/50 [00:48<00:01,  1.01s/it]

test_48: pred = 0 /// label = 0



100%|██████████| 50/50 [00:49<00:00,  1.02it/s]

test_49: pred = 1 /// label = 1

Accuracy: 66.00%
pred_list: [24, 11, 3, 12]
label_list: [11, 10, 2, 27]
correct_list: [10, 10, 1, 12]


In [23]:
# 다음 50개
newsClassifier2(question, labels_test_2, 1)

  2%|▏         | 1/50 [00:01<00:55,  1.13s/it]

test_50: pred = 0 /// label = 0



  4%|▍         | 2/50 [00:02<00:52,  1.10s/it]

test_51: pred = 1 /// label = 1



  6%|▌         | 3/50 [00:02<00:42,  1.10it/s]

...

test_97: pred = 0 /// label = 2



 98%|█████████▊| 49/50 [00:45<00:01,  1.09s/it]

test_98: pred = 1 /// label = 0



100%|██████████| 50/50 [00:46<00:00,  1.07it/s]

test_99: pred = 0 /// label = 2

Accuracy: 56.00%
pred_list: [26, 16, 6, 2]
label_list: [19, 11, 10, 10]
correct_list: [13, 11, 2, 2]


데이터와 label이 잘 정리된거로 나중에 또 테스트를 해봐야겠다. Gemma-2-2B로 테스트를 해보려 했으나 colab에서 매모리가 부족하다고 나와서 못 해본게 아쉽다. 나중에 EC2에서 돌려 볼 기회가 있으면 돌려봐야겠다.